# Let's chat with a friend and I have a brain

Demo chat with Leolani that combine face recognition and storage in the BRAIN.

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType

# specific imports
import uuid
from datetime import datetime
import cv2
import requests
import pickle

[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

    
    import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import intentions.talk as talk
import util.face_util as f_util
import intentions.get_to_know_you as friend

### Connect to the camera

In [3]:
### Link your camera
camera = cv2.VideoCapture(0)

### Initialise the BRAIN

In [4]:
# Initialise the brain in GraphDB
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-11-01 09:29:47,363 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted


2021-11-01 09:29:47.363 DEBUG basic_brain - __init__: Booted


2021-11-01 09:29:47,364 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain


2021-11-01 09:29:47.364 DEBUG basic_brain - clear_brain: Clearing brain


2021-11-01 09:29:49,362 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain


...

2021-11-01 09:29:52.767 DEBUG basic_brain - __init__: Booted


2021-11-01 09:29:53,012 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-11-01 09:29:53.012 DEBUG basic_brain - _submit_query: Posting query


2021-11-01 09:29:53,057 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


2021-11-01 09:29:53.057 INFO trust_calculator - compute_trust_network: Computed trust for all known agents


## Standard initialisation of a scenario

In [7]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-01-09:30:26  Created 
Directory  ../../data/2021-11-01-09:30:26/image  Created 


## Loading the docker containers for face detection and face property detection

#### If the docker images are running you can skip the next part

You only need to load the dockers once. The first time you load the docker, the images will be donwloaded from the DockerHub. This may take a few minutes depending on the speed of the internet connection. The images are cached in your local Docker installation.

One the images are in your local Docker, they are loaded instantaniously. Once the docker is started you do not need to start it again and you can skip the next commands.

In [8]:
#container_fdr = f_util.start_docker_container("tae898/face-detection-recognition:v0.1", 10002)
#container_ag = f_util.start_docker_container("tae898/age-gender:v0.2", 10003)

If there is a problem starting the dockers, you may need to kill them and start them again. Use the following command to kill and rerun the previous command. Note that if there are running already you should not restart. Starting it again gives an error that the port is occupied.

In [9]:
#!docker kill $(docker ps -q)

## Extracting and storing triples from an utterance

The next function processes any textSignal using the baseline language model from the Leolani platform.
This model uses a context-free grammar and closed-class lexicons specifically designed for social robot interaction.

The function creates a capsule for posting any triples and perspective to the BRAIN.
If no triples are extracted, a dummy prompt is returned: "Any gossip?".

Any thoughts coming from the BRAIN are returned.

## Create a new friend.

The functions in *intentions/get_to_know_you.py* are needed to get the properties and visual information for identifying a new friend.

The visual information is based on the camera images of the uses from which we extract an averaged embedding.
These embeddings are store in the *friend_embeddings* folder. 

By comparing an image with the stored embeddings, the system decides whether a person is a *stranger*.
In case the user is a *stranger*, the system will try to get to know him/her.

If you delete someone's embeddings from the *friend_embeddings* folder. This person will become a *stranger* again.

In [10]:
#First step is to identify the speaker

imagepath = ""
uuid_name = ""
# First signal
success, frame = camera.read()

if success:
    ### check if we know the human
    current_time = str(datetime.now().microsecond)
    imagepath = imagefolder + "/" + current_time + ".png"
    cv2.imwrite(imagepath, frame)
    imageSignal = d_util.create_image_signal(scenario, imagepath)
    (
        genders,
        ages,
        bboxes,
        faces_detected,
        det_scores,embeddings,
    ) = f_util.do_stuff_with_image(imagepath)

    # Here we assume that only one face is in the image
    # TODO: deal with multiple people.
    for k, (gender, age, bbox, uuid_name, faceprob, embedding) in enumerate(
        zip(genders, ages, bboxes, faces_detected, det_scores, embeddings)
    ):
        age = round(age["mean"])
        gender = "male" if gender["m"] > 0.5 else "female"
        bbox = [int(num) for num in bbox.tolist()]
    assert k == 0
    if uuid_name["name"] is None:
        ### This is a stranger
        ### We call the get_to_know function to create the data for a new friend
        HUMAN_ID, HUMAN_NAME, textSignal = friend.get_to_know_person(scenario, AGENT, gender, age, uuid_name, embedding)
        ### We store the data in the BRAIN for the human_id that is returned            
        name_thoughts, age_thoughts, gender_thoughts = friend.process_new_friend_and_think (scenario, 
                  place_id, 
                  location, 
                  HUMAN_ID,
                  textSignal,
                  imageSignal,
                  age,
                  gender,
                  HUMAN_NAME,
                  my_brain)
        
        ### The system responds to the processing of the new name input and stores it as a textsignal
        print(AGENT + f": Nice to meet you, {HUMAN_NAME}")
        response = f": Nice to meet you, {HUMAN_NAME}"
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)

    else:
        ### We know this person
        HUMAN_ID= uuid_name['name']
        HUMAN_NAME = HUMAN_ID.split("_t_")[0]
        response = f"Hi {HUMAN_NAME}. Nice to see you again :)"
        print(f"{AGENT}: {response}")
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)


2021-11-01 09:30:49.339 INFO face_util - load_binary_image: ../../data/2021-11-01-09:30:26/image/297874.png image loaded!
2021-11-01 09:30:50.115 INFO face_util - run_face_api: got <Response [200]> from server!...


TypeError: scalar() argument 1 must be numpy.dtype, not _IDProxy

## Proceed to communicate with an identified friend

The next while loop is for continuous communication with the identified user until *stop* or *bye*.
We process the user input and the captured image sequentially step by setp in the code block of the while loop.
At the end of the code block, a response is generated and new input from the user is requested.

In [11]:
### First prompt
response = "How are you doing "+HUMAN_NAME
textSignal = d_util.create_text_signal(scenario, response)
scenario.append_signal(textSignal)

print(AGENT + ": " + response)

#### Get the input from the user
utterance = input("\n")
print(HUMAN_NAME + ": " + utterance)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    # @TODO: also annotate the textSignal
    # Apply some processing to the textSignal and add annotations
        
    # We process the utterance and store triples in the brain
    # We catch the thoughts as the response
    thoughts = listen.process_text_and_think (scenario, 
                  place_id, 
                  location, 
                  textSignal, 
                  HUMAN_ID, my_brain)
    
    thoughts = talk.process_text_and_reply (scenario, place_id, location, HUMAN_ID, textSignal, replier, my_brain)
    print(AGENT + ": " + thoughts)
    textSignal = d_util.create_text_signal(scenario, thoughts)
    scenario.append_signal(textSignal)

    ## We capture the image again
    ## For now, we only take pictures of faces of the user
    ## @TODO add object recognition and check if there are other people detected than the user
   
    success, frame = camera.read()
    if success:
        current_time = str(datetime.now().microsecond)
        imagepath = imagefolder + "/" + current_time + ".png"
        cv2.imwrite(imagepath, frame)
        (
            genders,
            ages,
            bboxes,
            faces_detected,
            det_scores,
            embeddings,
        ) = f_util.do_stuff_with_image(imagepath)



    if success:
        imageSignal = d_util.create_image_signal(scenario, imagepath)
        container_id = str(uuid.uuid4())

        #### Properties are now stored as annotations
        #### We do not store these proeprties again to the BRAIN
        for gender, age, bbox, name, faceprob in zip(
            genders, ages, bboxes, faces_detected, det_scores
        ):

            age = round(age["mean"])
            gender = "male" if gender["m"] > 0.5 else "female"
            bbox = [int(num) for num in bbox.tolist()]

        f_util.add_face_annotation(imageSignal,container_id, "front_camera", container_id, current_time,
                                        bbox,HUMAN_ID,HUMAN_NAME, age, gender,faceprob)
                                   
        scenario.append_signal(imageSignal)
        
        ### We created a perceivedBy triple for this experience, 
        ### @TODO we need to include the bouding box somehow in the object
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "perceivedBy",
                                                                  imageSignal.id)

        response = my_brain.update(capsule, reason_types=True, create_label=False) 
         ## use this version if you want to use the URI as subject

    # Create the response from the system and store this as a new signal
    # We could use the throughts to respond
    # @TODO generate a response from the thoughts or based on the user query

    # Getting the next input signals
    utterance = input("\n")

    ### We now have a new input check if the user wants to continue

Leolani2: How are you doing Stranger



 I am great


Stranger: I am great
2021-11-01 09:31:13,863 -     INFO - cltl.triple_extraction.api.Chat (stranger)          000 - << Start of Chat with stranger >>


2021-11-01 09:31:13.863 INFO api - __init__: << Start of Chat with stranger >>


2021-11-01 09:31:13,865 -     INFO -               cltl.triple_extraction.api - Started POS tagger


...

2021-11-01 09:31:19,562 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query


2021-11-01 09:31:19.562 DEBUG basic_brain - _submit_query: Posting query


2021-11-01 09:31:19,613 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-11-01 09:31:19.613 DEBUG basic_brain - _submit_query: Posting query
2021-11-01 09:31:19.711 INFO face_util - load_binary_image: ../../data/2021-11-01-09:30:26/image/673663.png image loaded!
2021-11-01 09:31:20.479 INFO face_util - run_face_api: got <Response [200]> from server!...


TypeError: scalar() argument 1 must be numpy.dtype, not _IDProxy

### Set the end time of the scenario, save it and stop the containers

After we stopped the interaction, we set the end time and save the scenario as EMISSOR data.

In [14]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

In [15]:
### Stopping the docker containers
### This is only needed of you started them in this notebook

#f_util.kill_container(container_fdr)
#f_util.kill_container(container_ag)

In [16]:
#### Stop the camera when we are done
camera.release()

## End of notebook